In [63]:
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential

In [64]:
from tensorflow.keras.datasets import mnist
(X_train, y_train),(X_test, y_test) = mnist.load_data()

In [65]:
#Scale the independent features

X_train_scaled, X_test_scaled = X_train/255, X_test/255

In [66]:
X_train.shape, y_train.shape,X_test.shape, y_test.shape

((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

In [67]:
LAYERS = [Flatten(input_shape=(28,28), name='ip_layer'),
          Dense(30, activation=tf.keras.activations.relu, name='hidden_layer1'),
          Dense(30, activation=tf.keras.activations.relu, name='hidden_layer2'),
          Dense(10, activation=tf.keras.activations.softmax, name='op_layer')]

OPTIMIZER = tf.keras.optimizers.Adam(1e-3)

LOSS = tf.keras.losses.sparse_categorical_crossentropy

In [68]:
model_num_detector=Sequential(LAYERS)

In [69]:
model_num_detector.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 ip_layer (Flatten)          (None, 784)               0         
                                                                 
 hidden_layer1 (Dense)       (None, 30)                23550     
                                                                 
 hidden_layer2 (Dense)       (None, 30)                930       
                                                                 
 op_layer (Dense)            (None, 10)                310       
                                                                 
Total params: 24,790
Trainable params: 24,790
Non-trainable params: 0
_________________________________________________________________


In [70]:
# Note that the number of trainable params in "model_num_detector" is 24,790

In [71]:
model_num_detector.compile(optimizer = OPTIMIZER, loss = LOSS, metrics = ['accuracy'])

In [72]:
early_stop_cb = tf.keras.callbacks.EarlyStopping(restore_best_weights=True, patience=5)
model_num_detector.fit(X_train_scaled, y_train, validation_split=0.2, epochs=50, batch_size=32, verbose=True, callbacks=[early_stop_cb])

Epoch 1/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3751 - accuracy: 0.8883 - val_loss: 0.2013 - val_accuracy: 0.9426
Epoch 2/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.1815 - accuracy: 0.9475 - val_loss: 0.1636 - val_accuracy: 0.9529
Epoch 3/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.1438 - accuracy: 0.9560 - val_loss: 0.1345 - val_accuracy: 0.9599
Epoch 4/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.1188 - accuracy: 0.9644 - val_loss: 0.1283 - val_accuracy: 0.9609
Epoch 5/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.1004 - accuracy: 0.9699 - val_loss: 0.1196 - val_accuracy: 0.9653
Epoch 6/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.0896 - accuracy: 0.9727 - val_loss: 0.1137 - val_accuracy: 0.9656
Epoch 7/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.0809 - accuracy: 0.9751 - val_loss: 0.1174 - val_accuracy:

In [29]:
# Now lets use the same model's learning and try to build a new model to detect if the number detected is even or odd using TRANSFER LEARNING technique

In [73]:
model_num_detector.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 ip_layer (Flatten)          (None, 784)               0         
                                                                 
 hidden_layer1 (Dense)       (None, 30)                23550     
                                                                 
 hidden_layer2 (Dense)       (None, 30)                930       
                                                                 
 op_layer (Dense)            (None, 10)                310       
                                                                 
Total params: 24,790
Trainable params: 24,790
Non-trainable params: 0
_________________________________________________________________


In [74]:
for layer in model_num_detector.layers:
  print(layer.trainable)

True
True
True
True


In [75]:
for layer in model_num_detector.layers[:-1]:
  layer.trainable=False

In [76]:
for layer in model_num_detector.layers: # Skip the last layer and freeze the rest and create a new model that consumes it
  print(layer.trainable)

False
False
False
True


In [77]:
new_model=Sequential(layers=model_num_detector.layers[:-1])

In [78]:
new_model.add(Dense(1, activation=tf.keras.activations.sigmoid))

In [79]:
# Now preprocess the dataset to fit the new need for classifying odd/even number from the input

In [80]:
new_model.summary()

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 ip_layer (Flatten)          (None, 784)               0         
                                                                 
 hidden_layer1 (Dense)       (None, 30)                23550     
                                                                 
 hidden_layer2 (Dense)       (None, 30)                930       
                                                                 
 dense_13 (Dense)            (None, 1)                 31        
                                                                 
Total params: 24,511
Trainable params: 31
Non-trainable params: 24,480
_________________________________________________________________


In [81]:
y_train

array([5, 0, 4, ..., 5, 6, 8], dtype=uint8)

In [88]:
y_train_tmp=y_train.copy()

In [89]:
y_test

array([7, 2, 1, ..., 4, 5, 6], dtype=uint8)

In [90]:
y_train.shape, y_test.shape

((60000,), (10000,))

In [91]:
for idx, val in enumerate(y_train_tmp):
  if val%2==0:
    y_train_tmp[idx] = 1
  else:
    y_train_tmp[idx] = 0

In [92]:
y_train_tmp

array([0, 1, 1, ..., 0, 1, 1], dtype=uint8)

In [93]:
y_train

array([5, 0, 4, ..., 5, 6, 8], dtype=uint8)

In [94]:
for idx, val in enumerate(y_test):
  if val%2==0:
    y_test[idx] = 1
  else:
    y_test[idx] = 0

In [95]:
y_test

array([0, 1, 0, ..., 1, 0, 1], dtype=uint8)

In [97]:
new_model.compile(optimizer= OPTIMIZER, loss=tf.keras.losses.binary_crossentropy, metrics=['accuracy'])

In [99]:
new_model.fit(X_train, y_train_tmp, validation_split=0.2, epochs=50, batch_size=32, verbose = True, callbacks=[early_stop_cb])

Epoch 1/50
1500/1500 [==============================] - 3s 2ms/step - loss: 3.1108 - accuracy: 0.9520 - val_loss: 2.6985 - val_accuracy: 0.9503
Epoch 2/50
1500/1500 [==============================] - 3s 2ms/step - loss: 1.6262 - accuracy: 0.9514 - val_loss: 1.1586 - val_accuracy: 0.9536
Epoch 3/50
1500/1500 [==============================] - 3s 2ms/step - loss: 0.7449 - accuracy: 0.9479 - val_loss: 0.7082 - val_accuracy: 0.9427
Epoch 4/50
1500/1500 [==============================] - 3s 2ms/step - loss: 0.5242 - accuracy: 0.9419 - val_loss: 0.7262 - val_accuracy: 0.9179
Epoch 5/50
1500/1500 [==============================] - 3s 2ms/step - loss: 0.4444 - accuracy: 0.9422 - val_loss: 0.5123 - val_accuracy: 0.9442
Epoch 6/50
1500/1500 [==============================] - 4s 2ms/step - loss: 0.5115 - accuracy: 0.9401 - val_loss: 0.6601 - val_accuracy: 0.9317
Epoch 7/50
1500/1500 [==============================] - 3s 2ms/step - loss: 0.4384 - accuracy: 0.9420 - val_loss: 0.3985 - val_accuracy:

In [103]:
y_pred=(new_model.predict(X_test))>0.5

313/313 [==============================] - 1s 2ms/step


In [106]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.93      0.97      0.95      5074
           1       0.96      0.93      0.95      4926

    accuracy                           0.95     10000
   macro avg       0.95      0.95      0.95     10000
weighted avg       0.95      0.95      0.95     10000



In [111]:
new_model.evaluate(X_test, y_test)

313/313 [==============================] - 1s 2ms/step - loss: 0.3260 - accuracy: 0.9479


[0.32602670788764954, 0.9478999972343445]